In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import csv
pd.options.display.max_colwidth=500
from tqdm import tqdm

In [ ]:
import sys
sys.path.append('../')
from utils import *
from graphUtils import *

In [ ]:
import pickle
df = pickle.load(open('../../data/imdb/imdb_reviews_1000film.df','rb'))

In [ ]:
import datetime

movies_dic = {}
with open('../../data/imdb/imdb_movielens.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        if row[12].replace('_',' ') not in movies_dic: 
            movies_dic[row[12].replace('_',' ')] = []
            
        temp = [r.replace('_',' ') for r in row[0:10]]
        
        month,year = '',''
        if len(row[10]) > 0:        
            month = datetime.date(1900, int(row[10][4::]), 1).strftime('%B')
            year = row[10][0:4]
        
        temp.append(month.lower() + ' ' + year)
        temp.append(int(float(row[14])))
        
        movies_dic[row[12].replace('_',' ')].append(temp)
        

# Create Graph

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [ ]:
import networkx as nx

G=nx.Graph()
K = 3

i = 0
nodes_labels = {}
row_ids = {}
id_rows = {}

for movie in tqdm(movies_dic):
    i+=1
    row_name = str('RW'+str(i))
    G.add_node(row_name , label= row_name, type='Row')
    row_ids[row_name] = movie
    id_rows[movie] = row_name
    j=0
    
    #cols = [movie]
    cols = []
    for c in movies_dic[movie]: 
        for cl in c:
            cols.append(cl)
    
    for cl in cols:
        j+=1
        col_name = str('CL'+str(j))
        if cl == '': continue
        
        if not G.has_node(col_name):     G.add_node(col_name , label= col_name, type='Column')
        n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(str(normalize_text(cl)),K)]
        
        
        for tg in n_grams:
            
            if not G.has_node(tg):
                G.add_node(tg,label=tg, type='Token')
            G.add_edge(row_name,tg)
            G.add_edge(col_name,tg)
            
            
            

In [ ]:
i = 0
review_ids = {}
id_review = {}

for row in tqdm(df.itertuples()):
    if row.movie.lower() not in movies_dic: continue
    i += 1
    text = remove_stopwords(normalize_text(row.user_review.lower()))
    review_name = str('Review'+str(i))
    G.add_node(review_name , label= review_name, type='Review')
    review_ids[review_name] = row.user_review
    id_review[text] = review_name
    
    n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(text,K)]

    for tg in n_grams:
        
        
        if not G.has_node(tg):
            continue
           # G.add_node(tg,label=tg, type='Token')
            
        if not G.has_edge(review_name,tg):            G.add_edge(review_name,tg)
    

In [ ]:
ground_truth = {}
for row in tqdm(df.itertuples()):
    if row.movie.lower() not in movies_dic:         continue
    movie_name = row.movie.lower() 
    if movie_name not in ground_truth: ground_truth[movie_name] = []
    #if remove_stopwords(row.user_review.lower()) not in id_review: continue
    ground_truth[movie_name]. append(id_review[remove_stopwords(normalize_text(row.user_review.lower()))])

# Expansion with DBpedia

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
Q = """
    select distinct ?o    where {    
    <http://dbpedia.org/resource/%s> <http://dbpedia.org/ontology/wikiPageWikiLink> ?o . }
    """


def run_dbpedia(name):
    cname = ''
    for n in name.split('_'): cname += n.capitalize() + '_'
        
    cname = cname[0:-1]
    query = Q%cname
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    res = []
    for result in results["results"]["bindings"]:
        if not '.jpg' in result["o"]["value"] and not 'http://dbpedia.org/resource/Category:' in result["o"]["value"]:  
            res.append (result["o"]["value"].split('/')[-1])
    return res

In [ ]:
for node in tqdm(G.copy().nodes()):
    if G.nodes()[node]['type'] != 'Token': continue
    
    if len(node.split('_')) < 2: continue 
    
    if len(node.split('_'))==2 and node.split('_')[1].isdigit(): continue
    
    try:
        for e in run_dbpedia(node):
            for n in utils.normalize_text(e).split('_'):
                if not G.has_node(n):
                    G.add_node(n, label = n, type = 'Token')
                    G.add_edge(node,n,type= 'dbp')
    except:
        continue
        
for n in G.copy().nodes():
    if G.degree()[n] < 2:
        G.remove_node(n)

In [ ]:
#nx.write_graphml(G,'data/imdb/imdb_expanded.gml')

# Compression

## SSuM

In [ ]:
i = 0
node_ids = {}

for n in G.nodes:
    node_ids[n] = i
    i+=1
inv_nodes = {v: k for k, v in node_ids.items()}

In [ ]:
file = open('imdb_edgelist', 'w')

for e in G.edges():    file.write(str(node_ids[e[0]]) + '\t' + str(node_ids[e[1]]) + '\n')

In [ ]:
with open('../SSumM/output/summary_imdb_edgelist_0.9.txt') as f:
    sum_grapph = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
sum_grapph = [x.strip() for x in sum_grapph] 

In [ ]:
super_nodes,super_edges = {},[]
edge_weights = {}

for i in range(1,sum_grapph.index('<Superedge info>')):
    node = sum_grapph[i].split('\t')
    idd = node[0]
    node = [inv_nodes[int(n)] for n in node[1::]]
    super_nodes[idd] = node

for i in range(sum_grapph.index('<Superedge info>')+1,len(sum_grapph)):
    e = sum_grapph[i].split('\t')
    if e[0] not in edge_weights:        edge_weights[e[0]] = {}
    if e[1] not in edge_weights:        edge_weights[e[1]] = {}
        

    edge_weights[e[0]][e[1]] = e[2]
    edge_weights[e[1]][e[0]] = e[2]
    
    super_edges.append((e[0],e[1]))



In [ ]:
SG = nx.Graph()

for node in super_nodes:
    name = ''
    if ' '.join(super_nodes[node]).startswith(('RW','Review','CL')):
        name = ' '.join(super_nodes[node])
    else:
        name = super_nodes[node][0]
        
    SG.add_node(node , label= name, type='node')
    
for e in super_edges:
    SG.add_edge(e[0],e[1])

In [ ]:
G = SG
len(G.nodes()),len(G.edges())

# MSP

In [ ]:
#G1 = nx.Graph()
from random import choice
L = int(len(G.nodes())/2)
sp = []
i =0 
pbar = tqdm(total=L,position=0)
while i < L:
    first = choice([n for n in G.nodes() if G.nodes()[n]['type'] == 'Row'])
    second = choice([n for n in G.nodes() if G.nodes()[n]['type'] == 'Review'])
    paths = nx.all_shortest_paths(G, first,second, weight=None)
    for p in paths:
        G1.add_nodes_from(p)
        nx.add_path(G1,p)    
    i+=1
    pbar.update(1)
    

In [ ]:
G = G1
len(G.nodes()), len(G.edges())

# RandomWalk

In [ ]:
docs = []
random_paths = generate_random_walks(G,30,l=30)
for p in random_paths:
    docs.append(p)

# Word Embeddings

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm 
tagged_data = []
for d in tqdm(docs,position=0):
    tagged_data.append(word_tokenize(d))


In [ ]:
%env PYTHONHASHSEED=0
max_epochs = 10
vec_size = 300

model = Word2Vec(size=vec_size, min_count=10, window=3, sg=1, seed=0, workers = 4)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Model is Ready")

# Test

In [ ]:
movie_reviews = {}
for movie in tqdm(ground_truth):
    review_new = []
    
    m_id = id_rows[movie]
    
    for r in ground_truth[movie]:
        review_new.append(r)
        
    for r in random.sample(review_ids.keys(),98):
        if r not in ground_truth[movie]:
            review_new.append(r)
    
    
    
    movie_reviews[movie] = distance_w2v (model,m_id,review_ids,50000)

In [ ]:
for KK in [1,5,20,50000]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MR, hasP = 0,0,0

    for movie in movie_reviews:
        if movie not in ground_truth: continue
        i+=1
        preds = [f for (f,j) in movie_reviews[movie]][0:KK]
        golds = [f for f in ground_truth[movie]]

        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)